# Python API examples


## Creates

- createUser
- createMarket
- createTrade

### Create user

Creates a new user. Returns signature and verify keys.

~~~~
import requests
url = 'https://blocmarket.herokuapp.com/createUser'
headers = {'content-type': 'application/json'}
content = {}
response = requests.post(url, data=json.dumps(content), headers=headers)
~~~~

Example return as json:

~~~~
{'signingKey': '0cca0a3739eba76cc78823d9f6e898379014d8c53172f5e45c171a3c54a9f477',
 'traderId': '1',
 'verifyKey': 'cdcfb59431b2579a681cee65656cbed6f8d578d5cc30d3e759e86c1d3e3529ef'}
~~~~
        
### Create/update market

Creates/updates a market. Returns True and market details if successful. 

Specify:
- Market root and branch id
- Minimum and maximum outcomes
- Trader Id
- Sign and verify keys

Note that sending the private key is a terrible idea. In production signing is done on client side.

~~~~
sk = '0cca0a3739eba76cc78823d9f6e898379014d8c53172f5e45c171a3c54a9f477'
vk = 'cdcfb59431b2579a681cee65656cbed6f8d578d5cc30d3e759e86c1d3e3529ef'
tId = 2

url = 'https://blocmarket.herokuapp.com/createMarket'
headers = {'content-type': 'application/json'}
content_makemarket = {"signingKey": sk,
                    "traderId": tId, 
                    "verifyKey": vk,
                    "marketRootId": 1, 
                    "marketBranchId": 1, 
                    "marketMin": 0,
                    "marketMax":1}

response = requests.post(url, data=json.dumps(content_makemarket), headers=headers)
~~~~

Example return as json:

~~~~
{'allChecks': "{'inputChk': True, 'marketLimitChk': True, 'traderIdChk': True, 'marketId': '1', 'marketRangeChk': True, 'marketIndChk': True, 'sigChk': True, 'chainChk': True, 'ownerChk': False, 'timeChk': True}",
 'checks': 'True',
 'marketBranchId': 1,
 'marketId': 1,
 'marketMax': 1,
 'marketMin': 0,
 'marketRootId': 1,
 'traderId': 2}

~~~~
    
### Create trade

Create a trade. Returns True and trade details if successful.

Specify:

- Trader Id
- Market Id
- Price
- Quantity
- Sign and verify keys

~~~~
url = 'https://blocmarket.herokuapp.com/createTrade'
headers = {'content-type': 'application/json'}
content_maketrade = {"signingKey": sk,
                     "traderId": int(tId),
                     "verifyKey": vk,
                     "marketId": mkId,
                     "price": 0.55,
                     "quantity":1}
response = requests.post(url, data=json.dumps(content_maketrade), headers=headers)



~~~~

Example return as json:
~~~~

{'allChecks': "{'inputChk': True, 'traderIdChk': True, 'marketChk': True, 'sigChk': True, 'chainChk': True, 'timeChk': True, 'colChk': True}",
 'checks': 'True',
 'marketId': 1,
 'price': 0.551,
 'quantity': 1,
 'traderId': 1}


~~~~

## Views

- viewMarketBounds
- viewOrderBook
- viewOpenTrades
- viewMatchedTrades

### View markets

~~~~
url = 'https://blocmarket.herokuapp.com/viewMarketBounds'
headers = {'content-type': 'application/json'}
content = {}
response = requests.post(url, data=json.dumps(content), headers=headers)
~~~~

Example response as json:

~~~~
'{"marketId":{"0":1,"1":2,"2":3,"3":4},
"marketRootId":{"0":1,"1":2,"2":3,"3":4},
"marketBranchId":{"0":1,"1":1,"2":1,"3":1},
"marketMin":{"0":0.0,"1":0.0,"2":0.0,"3":0.0},
"marketMax":{"0":0.7,"1":0.7,"2":0.7,"3":0.7}}'
~~~~

### View full order book

~~~~
url = 'https://blocmarket.herokuapp.com/viewOrderBook'
headers = {'content-type': 'application/json'}
content = {'marketId': 1}
response = requests.post(url, data=json.dumps(content), headers=headers, stream=True)
~~~~

Example response as json:

~~~~
'{"marketId":{"0":1,"1":1},"price":{"0":0.55,"1":0.55},
"quantity":{"0":1.0,"1":-1.0,"traderId":{"0":1,"1":1},
"iMatched":{"0":true,"1":true,
"timeStampUTC":{"0":1547976585838,"1":1547976602766}}'
~~~~

Using a stream is only necessary if the order book is large and the request is truncated by the server, but it doesn't do any harm.

### View open trades

~~~~
url = 'https://https://blocmarket.herokuapp.com/viewOpenTrades'
content = {'marketId': 1}
response = requests.post(url, data=json.dumps(content), headers=headers, stream=True)
~~~~

~~~~
'{"marketId":{"12":1,"15":1,"18":1,"21":1,"24":1},
"price":{"12":0.43,"15":0.43,"18":0.43,"21":0.43,"24":0.43},
"quantity":{"12":-1.0,"15":-1.0,"18":-1.0,"21":-1.0,"24":-1.0},
"traderId":{"12":1,"15":1,"18":1,"21":1,"24":1},"timeStampUTC":
{"12":1547976742476,"15":1547976750344,"18":1547976758683,"21":1547976768968,"24":1547976775537}}'
~~~~

### View matched trades

~~~~
url = 'https://blocmarket.herokuapp.com/viewMatchedTrades'
headers = {'content-type': 'application/json'}
content = {'marketId': 1}
response = requests.post(url, data=json.dumps(content), headers=headers, stream=True)
~~~~

Example response as json:

~~~~
'{"marketId":{"0":1,"1":1},"price":{"0":0.43,"1":0.55},"quantity":{"0":0.0,"1":0.0},"traderId":{"0":1,"1":1}}'

~~~~

### View trader summary

A view joining order book and marketTable for a particular trader.

~~~~
url = 'https://blocmarket.herokuapp.com/viewTradeSummary'
headers = {'content-type': 'application/json'}
content = {'traderId': 2}
response = requests.post(url, data=json.dumps(content), headers=headers, stream=True)
~~~~

Example response as json:

~~~~
'{"marketId":{"0":1,"1":1},"price":{"0":0.8,"1":0.5},
"quantity":{"0":-1.0,"1":-1.0},"traderId":{"0":2,"1":2},
"iMatched":{"0":false,"1":true},
"timeStampUTC":{"0":1547982939316,"1":1547982938964},
"marketMin":{"0":0.0,"1":0.0},"marketMax":{"0":1.0,"1":1.0}}'
~~~~


# Python demonstration

In [1530]:
import requests
import json
import numpy as np
import pandas as pd
import time

%matplotlib inline
import matplotlib.pyplot as plt

# Running flask app from powershell
# $env:FLASK_APP = "api"
# $env:FLASK_ENV = "development"
# flask run

# Running flask from terminal on OSX
# export FLASK_APP=api
# export FLASK_ENV=development
# flask run

# Running with gunicorn (heroku, digitalocean, etc):
# Procfile >
# web: gunicorn wsgi --log-file -
# wsgi.py >
# from api import app as application
#
#if __name__ == "__main__":
#    application.run()

# https://devcenter.heroku.com/articles/getting-started-with-python
# https://www.digitalocean.com/community/tutorials/how-to-serve-flask-applications-with-gunicorn-and-nginx-on-ubuntu-14-04


In [1705]:
#baseurl = 'https://blocmarket.herokuapp.com/'
# baseurl = 'http://127.0.0.1:5000/'
baseurl = 'http://127.0.0.1:7000/'

In [1664]:
# Create a new user
url = baseurl +'createUser'
headers = {'content-type': 'application/json'}
content = {}
response = requests.post(url, data=json.dumps(content), headers=headers)

In [1665]:
# View response as json
response.json()

{'signingKey': 'b25c2f2b781b9057f7782cdf473b8908b7d80f5f3cf1af73fb7f852969f0610a',
 'traderId': '9',
 'verifyKey': 'af4e6327d832a47871259f754d90d1ea5ed019a2614479dd4677f05a03091306'}

In [1666]:
# Copy keys and traderid from response
sk = response.json()['signingKey']
vk = response.json()['verifyKey']
tId = int(response.json()['traderId'])


In [1667]:
# Create a new market
mkRtId = 1
mkBrId = 1


url = baseurl + 'createMarket'
content_makemarket = {"signingKey": sk,
                        "traderId": tId, 
                        "verifyKey": vk,
                        "marketRootId": 1, 
                        "marketBranchId": mkBrId, 
                        "marketMin": 0.11,
                        "marketMax":0.7}
# Post market
response = requests.post(url, data=json.dumps(content_makemarket), headers=headers)


In [1668]:
# response = requests.post(url, data=json.dumps(content_makemarket), headers=headers)
# View response as json
response.json()

{'allChecks': "{'inputChk': True, 'marketLimitChk': False, 'traderIdChk': True, 'marketId': '1', 'marketRangeChk': True, 'marketIndChk': True, 'sigChk': True, 'chainChk': True, 'ownerChk': False, 'timeChk': True}",
 'checks': 'False',
 'marketBranchId': 1,
 'marketId': 1,
 'marketMax': 0.7,
 'marketMin': 0.11,
 'marketRootId': 1,
 'traderId': 9}

In [1669]:
# Save market id from response
mkId = response.json()['marketId']


In [1670]:
# Create a trade
url = baseurl+'createTrade'

content_maketrade = {"signingKey": sk,
                     "traderId": tId,
                     "verifyKey": vk,
                     "marketId": mkId,
                     "price": 0.5512,
                     "quantity":1}
response = requests.post(url, data=json.dumps(content_maketrade), headers=headers)

In [1671]:
# View response as json
response.json()

{'allChecks': "{'inputChk': True, 'traderIdChk': True, 'marketChk': True, 'sigChk': True, 'chainChk': True, 'timeChk': True, 'colChk': True}",
 'checks': 'True',
 'marketId': 1,
 'price': 0.5512,
 'quantity': 1,
 'traderId': 9}

In [1672]:
# View response as a dataframe
pd.DataFrame(response.json(), index=[0])


,allChecks,checks,marketId,price,quantity,traderId
0,"{'inputChk': True, 'traderIdChk': True, 'marke...",True,1,0.5512,1,9


In [1673]:
mId = response.json()['marketId']

In [1674]:
# Create a matched trade
url = baseurl+'createTrade'
content_maketrade['quantity'] = -1
response = requests.post(url, data=json.dumps(content_maketrade), headers=headers)

In [1675]:
# View
response.json()

{'allChecks': "{'inputChk': True, 'traderIdChk': True, 'marketChk': True, 'sigChk': True, 'chainChk': True, 'timeChk': True, 'colChk': True}",
 'checks': 'True',
 'marketId': 1,
 'price': 0.5512,
 'quantity': -1,
 'traderId': 9}

In [1676]:
# Create many matched trades
url = baseurl+'createTrade'

for i in range(5):
    t = time.time()
    content_maketrade['quantity'] = -1
    response = requests.post(url, data=json.dumps(content_maketrade), headers=headers)
    content_maketrade['quantity']=1
    response = requests.post(url, data=json.dumps(content_maketrade), headers=headers)
    print('matched trade: ' + str(i) + ' in ' + str(time.time() - t) + ' seconds.')


matched trade: 0 in 4.0611677169799805 seconds.
matched trade: 1 in 4.177212953567505 seconds.
matched trade: 2 in 3.74184513092041 seconds.
matched trade: 3 in 4.136361122131348 seconds.
matched trade: 4 in 4.2492780685424805 seconds.


In [1677]:
content_maketrade

{'signingKey': 'b25c2f2b781b9057f7782cdf473b8908b7d80f5f3cf1af73fb7f852969f0610a',
 'traderId': 9,
 'verifyKey': 'af4e6327d832a47871259f754d90d1ea5ed019a2614479dd4677f05a03091306',
 'marketId': 1,
 'price': 0.5512,
 'quantity': 1}

In [1678]:
# Create many markets
url = baseurl+'createMarket'

for i in range(10):
    t = time.time()

    content_makemarket = {"signingKey": sk,
      "traderId": int(tId), "verifyKey": vk,
      "marketRootId": i+1, "marketBranchId": 1, "marketMin": 0, "marketMax":0.7}

    response = requests.post(url, data=json.dumps(content_makemarket), headers=headers)
    if response.json()['checks'] == 'True':
        print('market created: ' + str(i+1) + ' in ' + str(time.time() - t) + ' seconds.')
    else:
        print('failed to create market ' +str(i+1))
    

failed to create market 1
failed to create market 2
failed to create market 3
failed to create market 4
failed to create market 5
failed to create market 6
failed to create market 7
failed to create market 8
failed to create market 9
failed to create market 10


In [1679]:
response.json()

{'allChecks': "{'inputChk': True, 'marketLimitChk': False, 'traderIdChk': True, 'marketId': '35', 'marketRangeChk': True, 'marketIndChk': True, 'sigChk': True, 'chainChk': True, 'ownerChk': True, 'timeChk': True}",
 'checks': 'False',
 'marketBranchId': 1,
 'marketId': 35,
 'marketMax': 0.7,
 'marketMin': 0,
 'marketRootId': 10,
 'traderId': 9}

In [1680]:
# Create many sub-markets
url = baseurl+'createMarket'
marketRootId = tId

for i in range(10):
    t = time.time()

    content_makemarket = {"signingKey": sk,
      "traderId": int(tId), "verifyKey": vk,
      "marketRootId": marketRootId, "marketBranchId": 1+i, "marketMin": 0, "marketMax":0.7}

    response = requests.post(url, data=json.dumps(content_makemarket), headers=headers)
    if response.json()['checks'] == 'True':
        print('market created: ' + str(i+1) + ' in ' + str(time.time() - t) + ' seconds.')
    else:
        print('failed to create sub market ' +str(i+1))
    

failed to create sub market 1
failed to create sub market 2
failed to create sub market 3
failed to create sub market 4
failed to create sub market 5
failed to create sub market 6
failed to create sub market 7
failed to create sub market 8
failed to create sub market 9
failed to create sub market 10


In [1681]:
response.json()

{'allChecks': "{'inputChk': True, 'marketLimitChk': False, 'traderIdChk': True, 'marketId': '35', 'marketRangeChk': True, 'marketIndChk': True, 'sigChk': True, 'chainChk': True, 'ownerChk': True, 'timeChk': True}",
 'checks': 'False',
 'marketBranchId': 10,
 'marketId': 35,
 'marketMax': 0.7,
 'marketMin': 0,
 'marketRootId': 9,
 'traderId': 9}

In [1746]:
# View maximum and minimum of each market
url = baseurl+'viewMarketBounds'
content = {}
response = requests.post(url, data=json.dumps(content), headers=headers)


In [1747]:
pd.read_json(response.json())


,marketId,marketRootId,marketBranchId,marketMin,marketMax,marketDesc
0,1,1,1,0,1,"India v Australia Sun 24 Feb, 13:30 betfair ma..."
1,2,2,1,2500,4500,BTCUSD


In [1751]:
# View whole order book
url = baseurl+'viewOrderBook'
content = {'marketId': 2}
response = requests.post(url, data=json.dumps(content), headers=headers, stream=True)

In [1752]:
pd.read_json(response.json())


,marketId,price,quantity,traderId,iMatched,timeStampUTC
10,2,3616.39,1,2,True,2019-02-16 10:51:20.532
100,2,3618.66,-1,2,True,2019-02-16 12:01:28.328
101,2,3620.66,-1,2,True,2019-02-16 11:58:28.219
102,2,3620.66,1,2,True,2019-02-16 12:01:28.427
103,2,3620.39,1,2,False,2019-02-16 12:01:31.349
104,2,3622.39,-1,2,False,2019-02-16 12:01:31.456
11,2,3614.42,1,2,True,2019-02-16 10:51:23.023
12,2,3614.42,-1,2,True,2019-02-16 10:54:23.230
13,2,3616.42,-1,2,True,2019-02-16 10:51:23.120
14,2,3616.42,1,2,True,2019-02-16 10:54:23.322


In [1686]:
content_makemarket

{'signingKey': 'b25c2f2b781b9057f7782cdf473b8908b7d80f5f3cf1af73fb7f852969f0610a',
 'traderId': 9,
 'verifyKey': 'af4e6327d832a47871259f754d90d1ea5ed019a2614479dd4677f05a03091306',
 'marketRootId': 9,
 'marketBranchId': 10,
 'marketMin': 0,
 'marketMax': 0.7}

In [1687]:
# Create many unmatched trades
url = baseurl+'createTrade'

for i in range(5):
    t = time.time()
    content_maketrade['quantity'] = -1
    content_maketrade['price'] = 0.43
    response = requests.post(url, data=json.dumps(content_maketrade), headers=headers)
    print('unmatched trade: ' + str(i) + ' in ' + str(time.time() - t) + ' seconds.')


unmatched trade: 0 in 1.8239548206329346 seconds.
unmatched trade: 1 in 1.7473399639129639 seconds.
unmatched trade: 2 in 1.9287259578704834 seconds.
unmatched trade: 3 in 1.80330491065979 seconds.
unmatched trade: 4 in 1.7138910293579102 seconds.


In [1688]:
a = response.json()
a

{'allChecks': "{'inputChk': True, 'traderIdChk': True, 'marketChk': True, 'sigChk': True, 'chainChk': True, 'timeChk': True, 'colChk': True}",
 'checks': 'True',
 'marketId': 1,
 'price': 0.43,
 'quantity': -1,
 'traderId': 9}

In [1726]:
# View all open (unmatched trades)
url = baseurl+'viewOpenTrades'

content = {'marketId': mkId}
response = requests.post(url, data=json.dumps(content), headers=headers)

In [1727]:
pd.read_json(response.json())

,marketId,price,quantity,traderId,timeStampUTC
0,1,3647.09,1,1,2019-02-09 23:00:43.006


In [1728]:
# View matched trades (summed over marketId/price/traderId)
url = baseurl+'viewMatchedTrades'
content = {'marketId': mkId}
response = requests.post(url, data=json.dumps(content), headers=headers, stream=True)

In [1729]:
pd.read_json(response.json())

,marketId,price,quantity,traderId
0,1,3617.66,0,1
1,1,3618.27,0,1
10,1,3622.36,0,1
100,1,3638.49,0,1
101,1,3638.56,0,1
102,1,3638.78,0,1
103,1,3639.78,0,1
104,1,3640.18,0,1
105,1,3640.48,0,1
106,1,3640.60,0,1


In [1693]:
# Create many matched trades (own)
url = baseurl+'createTrade'

for i in range(5): 
    t = time.time()
    content_maketrade['quantity'] = -1
    response = requests.post(url, data=json.dumps(content_maketrade), headers=headers)
    content_maketrade['quantity']=1
    response = requests.post(url, data=json.dumps(content_maketrade), headers=headers)
    print('matched trade: ' + str(i) + ' in ' + str(time.time() - t) + ' seconds.')


matched trade: 0 in 3.894613027572632 seconds.
matched trade: 1 in 3.630260944366455 seconds.
matched trade: 2 in 3.622703790664673 seconds.
matched trade: 3 in 3.7874691486358643 seconds.
matched trade: 4 in 3.651853084564209 seconds.


In [1736]:
# View summary of trader for a particular trader
url = baseurl+'viewTradeSummary'
content = {'traderId': int(tId)}
response = requests.post(url, data=json.dumps(content), headers=headers, stream=True)

In [1737]:
pd.read_json(response.json())

,price,quantity,traderId,iMatched,timeStampUTC,marketId,marketRootId,marketBranchId,marketMin,marketMax,marketMinOutcome,marketMaxOutcome


In [1732]:
# Get a signed UTC timestamp
url = baseurl+'getSignedUTCTimestamp'
content = {}
response = requests.get(url, data=json.dumps(content), headers=headers)


In [1733]:
response.json()

{'timeStampUTC': '2019-02-16 07:49:04.637195',
 'timeStampUTCSignature': 'b\'\\x17\\x86\\xf8@\\x94\\xd9j\\xec\\xcfe\\x84\\xe1s\\\'F~q\\xf0MQ\\x92\\xeaqLU\\xe5\\x8c}K"\\xc0L\\x19\\x8e.\\xbe\\x8f\\x0er\\xe1\\x18e\\xba\\xa2\\xec\\xf6\\x00%\\x9e\\xc7N\\x02P\\xc8\\xc2\\xfd\\xc9\\x16\\x14\\x89/\\x845\\r\'',
 'verifyKey': 'cdcfb59431b2579a681cee65656cbed6f8d578d5cc30d3e759e86c1d3e3529ef'}


# Curl API examples



## Create user

~~~~
curl --header "Content-Type: application/json" -X POST http://blocmarket.herokuapp.com/createUser
~~~~
        
## Create market

~~~~
curl --data '{"signingKey_hex": "ece2efc138c8298d43caba1315ceda614e20644c74d46fed37871c47ea19afdf",
"traderId": 1, 
"verifyKey_hex": "9690a2e12971ae452d68bf3d08405090d45791533cf80740fd186aea4b6773fc",
"marketRootId": 1, 
"marketBranchId": 5, 
"marketMin": 0, 
"marketMax": 1}' 
--header "Content-Type: application/json" -X POST http://blocmarket.herokuapp.com/createMarket
~~~~
    
## Create trade

~~~~
curl --data '{"signingKey_hex": "ece2efc138c8298d43caba1315ceda614e20644c74d46fed37871c47ea19afdf",
"traderId": 1, 
"verifyKey_hex": "9690a2e12971ae452d68bf3d08405090d45791533cf80740fd186aea4b6773fc",
"marketId": 1,
"price": 0.1212,
"quantity": 1}' --header "Content-Type: application/json" -X POST http://blocmarket.herokuapp.com/createTrade

~~~~
        
## View order book (output to file)

~~~~
curl --header "Content-Type: application/json" -X POST http://blocmarket.herokuapp.com/viewOrderBook --output ob.txt
~~~~    
        
Also can load json from file with something like 
~~~~ 
curl -d "@data.json" -X POST http://blocmarket.herokuapp.com/data 
~~~~


# Matlab API examples


## Create user

~~~~
resp = webread('http://blocmarket.herokuapp.com/createUser')

~~~~

Returns something like

~~~~
'{"signingKey": '57f611e7cb7c5d4123be69663ccd06442c64db1a4f1161399dbb1b9bb6cc0e06'
           "traderId": '2'
      "verifyKey": 'ddaa5b0119a4ccfeffa36be0e6aac3682f798d021d5ded1d90512f2226e727b5'}'
~~~~

## Create market

~~~~
data = struct('signingKey', signingKey,...
'traderId', tId, ...
'verifyKey', verifyKey ,...
'marketRootId', 2, ...
'marketBranchId', 1, ...
'marketMin', 0.01, ...
'marketMax', 1)
resp = webwrite('http://blocmarket.herokuapp.com/createMarket', data, weboptions('RequestMethod', 'post','MediaType', 'application/json'))
~~~~

Returns something like

~~~~
'{
             "checks": 'True'
     "marketBranchId": 1
          "marketMax": 1
          "marketMin": 0.0100
       "marketRootId": 2
           "traderId": 1
}'
~~~~
## Create trade

~~~~
data = struct('signingKey', signingKey,...
'traderId', tId,...
'verifyKey', verifyKey ,...
'marketId', 1,...
'price', 0.51,...
'quantity', 1)
resp = webwrite('http://blocmarket.herokuapp.com/createTrade', data, weboptions('MediaType', 'application/json'))
~~~~

Returns something like
~~~~
'{
       "checks": 'True'
     "marketId": 1
        "price": 0.5100
     "quantity": 1
     "traderId": 1
 '}S
~~~~

## View market bounds
~~~~
data = struct('marketId', 1)
resp = webwrite('http://blocmarket.herokuapp.com/viewMarketBounds', data, weboptions('RequestMethod', 'post','contentType', 'json', 'Timeout', 10))
~~~~

## View order book
~~~~
data = struct('marketId':1)
resp = webwrite('http://blocmarket.herokuapp.com/viewOrderBook', data, weboptions('RequestMethod', 'post','contentType', 'json'), 'Timeout', 10)
~~~~

## View trade summary

~~~~
data = struct('traderId',1);
resp = webwrite('http://blocmarket.herokuapp.com/viewTradeSummary', data, weboptions('RequestMethod', 'post','MediaType', 'application/json', 'Timeout', 10))
~~~~


# Powershell

TODO


    

test cases with mismatched traderId/verifyKey
views to specific market
javascript/powershell for api
enforce max decimals DONE
table based checkcollateral
put up on digitalocean

write up another layer of trading interface